In [1]:
%matplotlib inline

In [ ]:
%%html
<style>
div.input {
    display:none;
}
</style>

In [2]:
import warnings
import os
import pandas as pd
import numpy as np
from datetime import timedelta
from ggplot import *

from plotly.graph_objs import Bar, Scatter, Figure, Layout

import plotly.plotly as py
import plotly.graph_objs as go



warnings.filterwarnings('ignore')

##############################################################
# Let's load the data

# All Invoicing data (2013-01-01 - 2015-01-01) DTN Destination Travel Network
data = pd.read_csv('onsite_ad_analysis/all_advertisers_v2.csv')
# All bookingcom data (2015-01-01 - 2016-08-31) Bookings from our site
files = os.listdir('onsite_ad_analysis')
booking_files = [f for f in files if 'booking' in f]
booking_frames = []
for f in booking_files:
    directory = 'onsite_ad_analysis'
    path = os.path.join(directory, f) 
    temp = pd.read_csv(path)
    booking_frames.append(temp)

booking_data = pd.concat(booking_frames)

##############################################################
# Let's format the DTN data

data = data.rename(
        columns={
        "Desc" : "advertiser_name"
    })

# Format Dates
data['Date'] = pd.to_datetime(data['Month-Year'])
data['Date'] = data['Date']



# Clean Descriptions (member names)
data['advertiser_name'] = data['advertiser_name'].str.replace(' - Monterey','')

data['advertiser_name'] = data['advertiser_name'].str.replace(' - April','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' - May','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' - June','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' - July','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' - August','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' - September','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' - October','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' - November','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' - December','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' - January','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' - February','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' - March','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' -April','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' -May','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' -June','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' -July','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' -August','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' -September','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' -October','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' -November','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' -December','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' -January','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' -February','')
data['advertiser_name'] = data['advertiser_name'].str.replace(' -March','')

# Format new column for month-year. Readability

data['Month-Year'] = data['Date'].apply(lambda x: str(x.month) + "-" + str(x.year))
data['Month'] = data['Date'].apply(lambda x: x.month)
data['Year']  = data['Date'].apply(lambda x: x.year)
data['Amount'] = data['Amount'].str.replace(' ', '')
data['Amount'] = data['Amount'].str.replace(',', '')
data['Amount'] = data['Amount'].apply(float)



##############################################################
# Let's format the Booking data

#Rename Columns
booking_data = booking_data.rename(
        columns={
        "book nr." : "book_nr",
        "booked"  : "booked",
        "arrival" : "arrival",
        "departure" : "departure",
        "comission_eur" : "comission_eur",
        "perc" : "fee_eur",
        "total transaction value ( EUR )" : "total_trans_value_eur",
        "slip nr." : "slip_nr",
        "affiliate id" : "affiliate_id",
        "hotel name" : "hotel_name",
        "accomodation type" : "accomodation_type",
        "hotel city" : "hotel_city",
        "hotel ufi" : "hotel_ufi"
    })


#Subset Columns needed
booking_data_v2 = booking_data[['booked',
                                'arrival',
                                'departure',
                                'total_trans_value_eur',
                                'hotel_name',
                                'hotel_city',
                                'accomodation_type']]


booking_data_v2['booked'] = pd.to_datetime(booking_data_v2['booked'])
booking_data_v2['arrival'] = pd.to_datetime(booking_data_v2['arrival'])
booking_data_v2['departure'] = pd.to_datetime(booking_data_v2['departure'])


data_v1 = data[data['Date'] < '2016-08-31']
data_v1 = data_v1[['advertiser_name','Amount','Month-Year','Month','Year','Date']]


##############################################################
# Let's transform the DTN data for visualization & modeling

## Top Spenders
ts = data_v1.groupby(['advertiser_name','Year'])
ts = ts['Amount'].sum()
ts = ts.reset_index()
ts = ts.sort(['Year','Amount'], ascending = False)
grouped = ts.groupby('Year')['Amount'].nlargest(3)
ts = ts.reset_index()
grouped_v2 = grouped.reset_index()
index = grouped_v2['level_1']
ts_v2 = ts[ts['index'].isin(index)]
ts_v3 = ts_v2[['advertiser_name', 'Year', 'Amount']]
ts_v3 = ts_v3.rename(columns={'advertiser_name': 'Member', 'Amount':'Spent'})
ts_v3

## Average Participating Member Contract Length Over Time
ct = data_v1.groupby(['Year'])
ct = ct['Month'].aggregate(len)
ct = ct.reset_index()
ct_v2 = ct.groupby(['Year'])['Month'].aggregate(np.mean)
ct_v2 = ct_v2.reset_index()
ct_v2['Year'] = ct_v2['Year'].apply(str)

## Revenue by Year
yr_revenue = data_v1.groupby(['Year'])['Amount'].aggregate(sum)
yr_revenue = yr_revenue.reset_index()
yr_revenue['Year'] = pd.to_datetime(yr_revenue['Year'], format = '%Y')
#yr_revenue['Year'] = yr_revenue['Year'].apply(lambda x: x.year)


# Revenue Over Month
revenue = data_v1.groupby(['Month-Year'])#.aggregate(sum)
revenue = revenue[['Amount']].aggregate(sum)
revenue = revenue.reset_index()
revenue['Month-Year'] = pd.to_datetime(revenue['Month-Year'])
revenue = revenue.sort('Month-Year')

# So far we haven't touched the Gross Amounts. We know need to account for the fact that the 
# CVB only gets 40% of the Gross Amount. 
revenue['MCCVB Amount'] = revenue['Amount']*.4

#Cumulative Revenue
revenue['Cumulative Revenue'] = revenue['Amount'].cumsum()
revenue['Cumulative MCCVB Revenue'] = revenue['MCCVB Amount'].cumsum()
revenue['Date'] = pd.to_datetime(revenue['Month-Year'])


# Revenue by year

unique_data = data_v1.drop_duplicates('advertiser_name')
# Unique Participants over Time
users = data_v1.groupby(['Date','advertiser_name']).aggregate(len)
users = users.reset_index()
# Aggregate Monthly
users_v2 = users.groupby(['Date']).aggregate(len)
users_v2 = users_v2.reset_index()
users_v2 = users_v2[['Date','Amount']]
users_v2 = users_v2.rename(columns={"Amount":"Unique Members"})
users_v2['Date'] = pd.to_datetime(users_v2['Date'])
users_v2['Month-Year'] = users_v2['Date'].apply(lambda x: str(x.month) + "-" + str(x.year))
users_v2['Month-Year'] = pd.to_datetime(users_v2['Month-Year'])
# Aggregate Yearly
users_year = unique_data.groupby(['advertiser_name','Year']).aggregate(len)
users_year = users_year.reset_index()
users_year_v2 = users_year.groupby(['Year']).aggregate(sum)
users_year_v2 = users_year_v2.reset_index()
users_year_v2 = users_year_v2.rename(columns={"Amount":"Unique Members"})


# Revenue and Participants over Time
full_u_r = pd.merge(users_v2, revenue, on = ['Date','Month-Year'])
full_u_r['Cost-Per-Member'] = full_u_r['Amount']/full_u_r['Unique Members']

# Revenue and Participants Yearly
full_u_r_v2 = full_u_r
full_u_r_v2['Date'] = pd.to_datetime(full_u_r_v2['Date'])
full_u_r_v2['Year'] = users_v2['Date'].apply(lambda x: str(x.year))
full_u_r_v3 = full_u_r_v2.groupby(['Year']).aggregate(sum)
full_u_r_v3 = full_u_r_v3.reset_index()


# Retention Rates, Who stayed and who bailed



# Loyalty - Avg Contract Lengths

# Introducton

The Monterey County Convention & Visitors Bureau has a number of benefits for members. In early 2013 they expanded their offering, in addition to basic online advertising included with membership the Monterey County Convention & Visitors Bureau (MCCVB) provides a media kit for supplemental online advertising. This kit offers a number of solutions aimed at different types of exposure. All of these solutions cost less than $300 per month and start at $50 per month.

Seemonterey.com is a substantial source of online traffic and therefore impressions and engagement, averaging 1.6 million new users every year and boasting over 400,000 pageviews per month. The quality of the traffic is also worth noting with online visitors staying onsite longer and consuming our member's listings which have been viewed 250,000 times year to date. In addition to our impressions, our member websites have received 100,000 click throughs from seemonterey.com year to date.

This advertisement, while beneficial to our members, is being re-considered objectively in terms of its monetization. The following briefly summarizes the current state of our relationship with Destination Travel Network, our members usage of this program, and what other DMOs are doing. Finally we summarize our |Pros & Cons| Best Practices | POV.

Are we extracting more value in terms of stakeholder disagreement then providing value to our constituents?


## Context


Destination Travel Network (DTN) is our partner who provides the ability and support for members to advertise on the website. Since the inception of the media kit and the partnership with DTN, the program has brought in approximately VALUE$104,693VALUE directly from members. This represents the total amount spent by members for additional advertising online. The MCCVB has brought in VALUE$42,000VALUE from the exchange with the rest allocated towards DTN. The current monthly average for the last 3 months is VALUE$5,555VALUE with 21 members participating.

Shown below is our yearly revenue, please keep in mind this analysis is done through August. As you can see the program has been growing aggressively in the first two years and appears to be leveling off.




In [ ]:
#INSERT YEARLY BAR HERE #

## Yearly Plot
yr_plot_data = [
                go.Bar(
                x = yr_revenue['Year'],
                y = yr_revenue['Amount'])
]

#py.iplot(yr_plot_data, filename='bar-chart')

Shown below is a graph of the number of participating members overtime.


In [ ]:
#MEMBERS OVER TIME HERE#

# Unique Members
rev_um = ggplot(aes(x='Month-Year', y = 'Unique Members'), data = full_u_r)
rev_um = rev_um + geom_line() + ggtitle("Participation")
rev_um


Our top three spenders for three years of operation are shown below. Please note that the members who are no longer the top spenders are also still members, they were just outspent by later arrivals.


In [ ]:
#INSERT TOP SPENDERS HERE#
# Table of Top 3 Spenders Per Year
ts_v3

Our participating members number of contract months per year is for approximately over the past three years can be shown below. It is important to note this is only through August of this year, so 2016 isn't entirely represented. You can see the average increasing, with steady participation.

In [ ]:
#INSERT CONTRACT MONTHS HERE#

## Contract Months
yr_cm_data = [
                go.Scatter(
                x = ct_v2['Year'],
                y = ct_v2['Month'])
]

#py.iplot(yr_cm_data, filename='nm-contracts-chart')

## Other DMOs

We reached out to 7 of our partner DMOs to get a better understanding of current  practices and structures.Visit San Jose does offer paid advertisements on their website, but they are not membership based and do not require membership dues. Visit San Luis Obispo does not currently offer supplemental online advertising, however anticipates it will be able to offer it in the near future. Visit Santa Barabara does offer supplemental online advertising in conjunction with membership dues. SF Travel offers both supplemental online advertising and membership dues, among more offerings. Visit Ventura offers supplemental online advertising in conjunction with membership dues. We are awaiting more responses

Without more data, we will not be able to analyze DMO patterns. However, I believe it is reasonable for the reader to note that the size of the DMO will largely indicate the amount of offerings for a member. SF Travel has many packages and opportunities for different levels of engagement depending on the member. And that is the key... the diversity, or homogeniety, of the members will likely determine the type, complexity, and cost of offerings.


## POV



Members:


MCCVB:


Stakeholder: